In [1]:
import pandas as pd
import numpy as np
from math import floor
import math
import os
from datetime import datetime
import xarray as xr

# Listado de playas con avistamientos y fechas

In [2]:
avistamientos_df = pd.read_excel("./Physalia_Ambiental_R.xlsx")
# avistamientos_df = pd.read_excel("../Physala_Data/Datos_Physalia_20171010.xls")

columnas = avistamientos_df.iloc[0]

#Quito las 3 primeras filas, debido al formato de la excell
avistamientos_df = avistamientos_df.iloc[3:] 
avistamientos_df.columns = columnas

# Me quedo solo con los datos de avistamientos
avistamientos_df = avistamientos_df[["Latitud","Longitud","Año","Mes","Día","Avistamientos"]]

#Transdormaciones para sacar con fecha (datetime)
avistamientos_fecha_df=avistamientos_df[["Año","Mes","Día"]]
avistamientos_fecha_df.columns = ["year","month","day"]

fecha = pd.to_datetime(avistamientos_fecha_df)

avistamientos_df["Fecha"]=fecha
avistamientos_df=avistamientos_df[["Latitud","Longitud","Fecha","Avistamientos"]]
avistamientos_df.head()

,Latitud,Longitud,Fecha,Avistamientos
3,-25.4,-70.4833,2014-05-01,1
4,-18.4631,-70.3053,2014-05-01,1
5,-27.0658,-70.8259,2014-05-09,13
6,-18.4933,-70.3261,2014-05-10,15
7,-27.122,-70.8576,2014-05-10,23


# Exporto listado de playas sin redondear coordenadas

In [3]:
# Genera excel con las coordenadas de las playas
playas_df = avistamientos_df.reset_index()
playas_df = playas_df[['Latitud','Longitud']]
playas_df.to_excel('./playasSinRedondeoCoord.xlsx')

In [28]:
base = 1/4
def redondeo(x,y):
    return  [base * round(x/base),base * round(y/base)]
x = redondeo(36.7663638888889,-73.1697694444445)
x

[36.75, -73.25]

# Redondeo a cada cuarto de grado
Redondeo de latitud y longitud para juntar lecturas de una misma playa con coordenadas muy proximas

Se exporta el dataframe generado a un excel

In [4]:
base=1/4
def floorQuarter(x):
    return (floor(x * 4) / 4.0) 

def redondeo(x):
    return  base * round(x/base)

#Se añaden atributos con la longitud y latitud redondeadas a cada cuarto de grado
avistamientos_df["Lat_floor"] = avistamientos_df.Latitud.map(redondeo)
avistamientos_df["Long_floor"] = avistamientos_df.Longitud.map(redondeo)

df = avistamientos_df[['Lat_floor', 'Long_floor',"Fecha","Avistamientos"]]
df = df.set_index(['Lat_floor', 'Long_floor',"Fecha"])

#DataFrame total avistamientos de playas en esa cuadricula
df_sum = df.groupby(['Lat_floor', 'Long_floor',"Fecha"]).sum()

#DataFrame número de playas en esa cuadricula
df_count = df.groupby(['Lat_floor', 'Long_floor',"Fecha"]).count()

#Dataframe con el total de avistamientos y el número de playas
df_join = df_sum.join(df_count,lsuffix="I",rsuffix="R")
df_join.columns=['Suma', 'N_Playas']
df_join['Media_Playas'] = df_join.Suma /df_join.N_Playas

# DataFrame de avistamientos procesado
df_join.to_excel("AvistamientosConRedondeoCoord-025.xlsx")

df_join.head()

Suma  N_Playas  Media_Playas
Lat_floor Long_floor Fecha                                   
-42.75    -74.00     2015-05-27     1         1           1.0
-41.75    -73.75     2014-06-04     1         1           1.0
                     2015-06-10    20         1          20.0
                     2015-06-11   151         1         151.0
          -73.50     2015-05-23     1         1           1.0

# Redondeo con salto entre cuadrantes de dataSet original (0,0833)
Redondeo de latitud y longitud para juntar lecturas de una misma playa con coordenadas muy proximas

Se exporta el dataframe generado a un excel

In [23]:
base=1/12
def floorTwelfth(x):
    return (floor(x * 12) / 12.0)

def redondeo(x):
    return  base * round(x/base)

#Se añaden atributos con la longitud y latitud redondeadas a cada cuarto de grado
avistamientos_df["Lat_floor"] = avistamientos_df.Latitud.map(redondeo)
avistamientos_df["Long_floor"] = avistamientos_df.Longitud.map(redondeo)

df = avistamientos_df[['Lat_floor', 'Long_floor',"Fecha","Avistamientos"]]
df = df.set_index(['Lat_floor', 'Long_floor',"Fecha"])

#DataFrame total avistamientos de playas en esa cuadricula
df_sum = df.groupby(['Lat_floor', 'Long_floor',"Fecha"]).sum()

#DataFrame número de playas en esa cuadricula
df_count = df.groupby(['Lat_floor', 'Long_floor',"Fecha"]).count()

#Dataframe con el total de avistamientos y el número de playas
df_join = df_sum.join(df_count,lsuffix="I",rsuffix="R")
df_join.columns=['Suma', 'N_Playas']
df_join['Media_Playas'] = df_join.Suma /df_join.N_Playas

# DataFrame de avistamientos procesado
df_join.to_excel("AvistamientosConRedondeoCoord-00833.xlsx")

In [24]:
df_join.head()

Suma  N_Playas  Media_Playas
Lat_floor  Long_floor Fecha                                   
-42.666667 -74.083333 2015-05-27     1         1           1.0
-41.833333 -73.833333 2014-06-04     1         1           1.0
           -73.500000 2015-05-23     1         1           1.0
-41.666667 -73.750000 2015-06-10    20         1          20.0
                      2015-06-11   151         1         151.0

## Exporto listado de coordenadas para utilixarlo en la web

In [25]:
df_web = df_join.reset_index().rename(columns={"Lat_floor": "Latitud", "Long_floor": "Longitud"})[['Latitud','Longitud']]
df_web.to_excel("Playas.xlsx")

# Estructura Buena

In [8]:
# DF con playas
# df_playas = pd.read_excel('./AvistamientosConRedondeoCoord-025.xlsx').reset_index()
df_playas = pd.read_excel('./AvistamientosConRedondeoCoord-00833.xlsx').reset_index()
'''
Las celdas que estaban unidas por tener el mismo valor, tienen valores nulos
por lo que las relleno con us valor correspodiente
'''
df_playas = df_playas.fillna(method= 'ffill',axis=0)
# Cojo las columnas que me interesan
df_playas = df_playas[["Suma","Lat_floor","Long_floor","Fecha"]]
# Renombro las columnas 
df_playas = df_playas.rename(columns={"Suma": "Avistamientos", "Lat_floor": "Latitud", "Long_floor": "Longitud"})
# Ordeno por fecha
df_playas = df_playas.sort_values(by=["Fecha"])

In [9]:
df_playas # DataFrame con avistamientos en cada playa
cuadrantes = { # Numero de cuadrantes que se concatenarán con las playas
    'izquierda': 3 ,
    'arriba': 0,
    'abajo': 0
}
salto = 1/12 # Salto de coordenadas entre los cuadrantes
listado_archivos = os.listdir('../../descargas') # Listo todos los archivos de Copernicus
variables = ['Coord','Profundidad','mlotst','zos','bottomT','thetao','so','uo','vo']

In [10]:
def busca_archivo(fecha):
    texto ='_{}_'.format(str(fecha).split()[0].replace('-',''))
    archivo = [x for x in listado_archivos if str(texto) in x]
    data = xr.open_dataset('../../descargas/{}'.format(archivo[0])) # cargo el archivo
    return data # devuelvo dataset

def dame_datos(latitud,longitud,ds):
    return ds.sel({'latitude':latitud,'longitude': longitud})

def comprueba_datos(latitud,longitud,ds):
    valor = dame_datos(latitud,longitud,ds)
    while math.isnan(valor.mlotst[0]):
        longitud = longitud - salto
        valor = dame_datos(latitud,longitud,ds)
    return latitud,longitud # devuelvo las coordenadas con datos

def crea_DF():
    df_resultados = None
    df_resultados = pd.DataFrame(columns=["Avistamientos","Latitud","Longitud","Fecha",'Coord',"Profundidad","mlotst",'zos','bottomT','thetao','so','uo','vo']) # columnas básicas
    for izquierda in range(cuadrantes['izquierda']): # por cada cuadrante a la izquierda meto un grupo de columnas
        var_aux = [str(x+'_izq_'+str(izquierda+1)) for x in variables]
        df_aux = pd.DataFrame(columns=var_aux)
        df_resultados = df_resultados.join(df_aux)
        for superior in range (cuadrantes['arriba']): # por cada cuadrante a la superior meto un grupo de columnas
            var_aux = [x+'_sup_'+str(izquierda+1)+'.'+str(superior+1) for x in variables]
            df_aux = pd.DataFrame(columns=var_aux)
            df_resultados = df_resultados.join(df_aux)
        for inferior in range (cuadrantes['abajo']): # por cada cuadrante a la inferior meto un grupo de columnas    
            var_aux = [x+'_inf_'+str(izquierda+1)+'.'+str(inferior+1) for x in variables]
            df_aux = pd.DataFrame(columns=var_aux)
            df_resultados = df_resultados.join(df_aux)
    return df_resultados

def meto_en_df(df_resultados,row,coordenadas,df,indice,variables_actuales):
    for x in range(3):
        df_resultados.loc[indice + x,["Avistamientos","Latitud","Longitud","Fecha"]] = row.values.tolist() # meto en df_resulados los datos de ese cuadrante
        vars = df.loc[x,['depth', 'mlotst','zos','bottomT','thetao','so','uo','vo']].values.tolist()
        vars.insert(0,coordenadas)
        df_resultados.loc[indice + x,variables_actuales] = vars # meto coordenadas reales utilizadas y datos 
    return df_resultados

def dameCoordIzq(lat,long,n_saltos):
    return [lat,long-(salto*n_saltos)]

def dameCoordSuperior(lat,long,n_saltos):
    coordenadas = dameCoordIzq(lat,long,n_saltos)
    return [lat+(salto*n_saltos),long]

def dameCoordInferior(lat,long,n_saltos):
    return [lat-(salto*n_saltos),long]

In [11]:
def crea_estr():
    df_resultados = crea_DF() # creo df con columnas necesarias
    #print(df_resultados)
    for contador,[index, row] in enumerate(df_playas.iterrows()): # por cada playa
        print(contador,end='\r')
        ds = busca_archivo(row['Fecha']) # busco el archivo de la fecha 
        coordenadas = comprueba_datos(row['Latitud'],row['Longitud'],ds) # miro a ver si el cuadrante tiene datos, sino cojo el siguiente por la izquierda

        df = dame_datos(coordenadas[0],coordenadas[1],ds).to_dataframe().reset_index() # dataframe con los datso oceanicos de esas coordenadas y en esa fecha
        df_resultados = meto_en_df(df_resultados,row,coordenadas,df,contador * 3,variables)

        # datos de las celdas extras
        for izquierda in range(cuadrantes['izquierda']): # por cada cuadrante a la izquierda, añado datos
            coordenadas = dameCoordIzq(coordenadas[0],coordenadas[1],izquierda+1)
            df = dame_datos(coordenadas[0],coordenadas[1],ds).to_dataframe().reset_index() 
            var_aux = [x+'_izq_'+str(izquierda+1) for x in variables]
            df_resultados = meto_en_df(df_resultados,row,coordenadas,df,contador * 3,var_aux)
            coord_aux = coordenadas
            for superior in range (cuadrantes['arriba']): # por cada cuadrante a la superior, añado datos
                coord_aux = dameCoordSuperior(coord_aux[0],coord_aux[1],superior+1)
                df = dame_datos(coord_aux[0],coord_aux[1],ds).to_dataframe().reset_index() 
                var_aux = [x+'_sup_'+str(izquierda+1)+'.'+str(superior+1) for x in variables]
                df_resultados = meto_en_df(df_resultados,row,coord_aux,df,contador * 3,var_aux)
            coord_aux = coordenadas
            for inferior in range (cuadrantes['abajo']): # por cada cuadrante a la inferior, añado datos  
                coord_aux = dameCoordInferior(coord_aux[0],coord_aux[1],inferior+1)
                df = dame_datos(coord_aux[0],coord_aux[1],ds).to_dataframe().reset_index() 
                var_aux = [x+'_inf_'+str(izquierda+1)+'.'+str(inferior+1) for x in variables]
                df_resultados = meto_en_df(df_resultados,row,coord_aux,df,contador * 3,var_aux)

#         if contador == 5:
#             break
    return df_resultados

In [12]:
df_resultados = crea_estr()
df_resultados

,Avistamientos,Latitud,Longitud,Fecha,Coord,Profundidad,mlotst,zos,bottomT,thetao,...,vo_izq_2,Coord_izq_3,Profundidad_izq_3,mlotst_izq_3,zos_izq_3,bottomT_izq_3,thetao_izq_3,so_izq_3,uo_izq_3,vo_izq_3
0,1,-25.4167,-70.5,2014-05-01 00:00:00,"(-25.41666666666667, -70.58333333333333)",0.494025,12.5126,0.194708,12.5373,15.8861,...,-0.151372,"[-25.41666666666667, -71.08333333333333]",0.494025,15.8696,0.138249,1.3932,17.5597,34.8186,-0.14771,-0.0561541
1,1,-25.4167,-70.5,2014-05-01 00:00:00,"(-25.41666666666667, -70.58333333333333)",5.07822,12.5126,0.194708,12.5373,15.8524,...,-0.178228,"[-25.41666666666667, -71.08333333333333]",5.07822,15.8696,0.138249,1.3932,17.5487,34.8186,-0.151982,-0.0848415
2,1,-25.4167,-70.5,2014-05-01 00:00:00,"(-25.41666666666667, -70.58333333333333)",9.573,12.5126,0.194708,12.5373,15.8209,...,-0.188604,"[-25.41666666666667, -71.08333333333333]",9.573,15.8696,0.138249,1.3932,17.5341,34.8186,-0.151982,-0.0970489
3,1,-18.5,-70.3333,2014-05-01 00:00:00,"(-18.5, -70.33333333333333)",0.494025,10.5289,0.219428,15.3323,17.6198,...,-0.0311289,"[-18.5, -70.83333333333333]",0.494025,12.8178,0.209357,4.47533,21.8262,35.2428,0.121464,-0.16419
4,1,-18.5,-70.3333,2014-05-01 00:00:00,"(-18.5, -70.33333333333333)",5.07822,10.5289,0.219428,15.3323,17.2711,...,-0.0598163,"[-18.5, -70.83333333333333]",5.07822,12.8178,0.209357,4.47533,21.8152,35.2412,0.127567,-0.190435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1546,2,-26.4167,-70.6667,2016-03-13 00:00:00,"(-26.41666666666667, -70.66666666666667)",5.07822,7.32444,0.0567644,19.1696,19.5776,...,0.386975,"[-26.41666666666667, -71.16666666666667]",5.07822,11.7496,0.138859,1.54115,22.0386,34.8125,-0.04944,0.044557
1547,2,-26.4167,-70.6667,2016-03-13 00:00:00,"(-26.41666666666667, -70.66666666666667)",9.573,7.32444,0.0567644,19.1696,18.175,...,0.355235,"[-26.41666666666667, -71.16666666666667]",9.573,11.7496,0.138859,1.54115,21.9925,34.8094,-0.04944,0.0305185
1548,5,-27,-70.8333,2016-03-13 00:00:00,"(-27.0, -70.83333333333333)",0.494025,10.5289,0.0723289,13.1306,19.8596,...,0.0860622,"[-27.0, -71.33333333333333]",0.494025,10.5289,0.108951,1.36244,20.4829,34.6904,0.120853,0.191046
1549,5,-27,-70.8333,2016-03-13 00:00:00,"(-27.0, -70.83333333333333)",5.07822,10.5289,0.0723289,13.1306,19.6274,...,0.0592059,"[-27.0, -71.33333333333333]",5.07822,10.5289,0.108951,1.36244,20.354,34.6889,0.11475,0.162358


In [13]:
df_resultados.to_excel('EstructuraFinal.xlsx')


## Estructura sin rellenar huecos sin datos

In [14]:
def crea_estr():
    df_resultados = crea_DF() # creo df con columnas necesarias
    #print(df_resultados)
    for contador,[index, row] in enumerate(df_playas.iterrows()): # por cada playa
        print(contador,end='\r')
        ds = busca_archivo(row['Fecha']) # busco el archivo de la fecha 
        coordenadas = [row['Latitud'],row['Longitud']]

        df = dame_datos(row['Latitud'],row['Longitud'],ds).to_dataframe().reset_index() # dataframe con los datso oceanicos de esas coordenadas y en esa fecha
        df_resultados = meto_en_df(df_resultados,row,[row['Latitud'],row['Longitud']],df,contador * 3,variables)
        
        # datos de las celdas extras
        for izquierda in range(cuadrantes['izquierda']): # por cada cuadrante a la izquierda, añado datos
            coordenadas = dameCoordIzq(coordenadas[0],coordenadas[1],izquierda+1)
            df = dame_datos(coordenadas[0],coordenadas[1],ds).to_dataframe().reset_index() 
            var_aux = [x+'_izq_'+str(izquierda+1) for x in variables]
            df_resultados = meto_en_df(df_resultados,row,coordenadas,df,contador * 3,var_aux)
            coord_aux = coordenadas
            for superior in range (cuadrantes['arriba']): # por cada cuadrante a la superior, añado datos
                coord_aux = dameCoordSuperior(coord_aux[0],coord_aux[1],superior+1)
                df = dame_datos(coord_aux[0],coord_aux[1],ds).to_dataframe().reset_index() 
                var_aux = [x+'_sup_'+str(izquierda+1)+'.'+str(superior+1) for x in variables]
                df_resultados = meto_en_df(df_resultados,row,coord_aux,df,contador * 3,var_aux)
            coord_aux = coordenadas
            for inferior in range (cuadrantes['abajo']): # por cada cuadrante a la inferior, añado datos  
                coord_aux = dameCoordInferior(coord_aux[0],coord_aux[1],inferior+1)
                df = dame_datos(coord_aux[0],coord_aux[1],ds).to_dataframe().reset_index() 
                var_aux = [x+'_inf_'+str(izquierda+1)+'.'+str(inferior+1) for x in variables]
                df_resultados = meto_en_df(df_resultados,row,coord_aux,df,contador * 3,var_aux)
        
    return df_resultados

In [15]:
df_resultados_con_vacios = crea_estr()

In [16]:
df_resultados_con_vacios.head()

,Avistamientos,Latitud,Longitud,Fecha,Coord,Profundidad,mlotst,zos,bottomT,thetao,...,vo_izq_2,Coord_izq_3,Profundidad_izq_3,mlotst_izq_3,zos_izq_3,bottomT_izq_3,thetao_izq_3,so_izq_3,uo_izq_3,vo_izq_3
0,1,-25.4167,-70.5,2014-05-01 00:00:00,"[-25.41666666666667, -70.5]",0.494025,NaN,NaN,NaN,NaN,...,-0.237434,"[-25.41666666666667, -71.0]",0.494025,15.4118,0.142216,1.48183,17.1649,34.8079,-0.130009,-0.0195318
1,1,-25.4167,-70.5,2014-05-01 00:00:00,"[-25.41666666666667, -70.5]",5.07822,NaN,NaN,NaN,NaN,...,-0.26368,"[-25.41666666666667, -71.0]",5.07822,15.4118,0.142216,1.48183,17.1481,34.8079,-0.135502,-0.0482192
2,1,-25.4167,-70.5,2014-05-01 00:00:00,"[-25.41666666666667, -70.5]",9.573,NaN,NaN,NaN,NaN,...,-0.273446,"[-25.41666666666667, -71.0]",9.573,15.4118,0.142216,1.48183,17.1312,34.8079,-0.136723,-0.0616474
3,1,-18.5,-70.3333,2014-05-01 00:00:00,"[-18.5, -70.33333333333333]",0.494025,10.5289,0.219428,15.3323,17.6198,...,-0.0311289,"[-18.5, -70.83333333333333]",0.494025,12.8178,0.209357,4.47533,21.8262,35.2428,0.121464,-0.16419
4,1,-18.5,-70.3333,2014-05-01 00:00:00,"[-18.5, -70.33333333333333]",5.07822,10.5289,0.219428,15.3323,17.2711,...,-0.0598163,"[-18.5, -70.83333333333333]",5.07822,12.8178,0.209357,4.47533,21.8152,35.2412,0.127567,-0.190435


In [17]:
df_resultados_con_vacios.to_excel('EstructuraConVacios.xlsx')